# Outlier 처리하기

* 통계적으로 outlier, 특이치는 다른 관측치와 크게 다른 데이터 포인트를 말한다.
* 특이치는 통계 분석에 문제를 일으킬 수 있다.(평균과 표준편차에 영향을 줌)


### 이상치 확인하기
* [-1.5 * IQR ~ 1.5*IQR] 의 구간에서 벗어나 있는 경우
* 백분위 수에서 5th ~ 95th 범위에서 벗어나는 경우

In [1]:
import pandas as pd
import numpy as np

In [2]:
# outlier를 포함한 임의의 데이터
x = pd.Series([23,1,3,5,34,6,32,7,45,34,78])
x.describe()

count    11.000000
mean     24.363636
std      23.589289
min       1.000000
25%       5.500000
50%      23.000000
75%      34.000000
max      78.000000
dtype: float64

보통 통계적으로 [-1.5 * IQR ~ 1.5*IQR] 의 구간에서 벗어나 있는 경우를 outlier라고 한다.

In [3]:
# 이상치의 인덱스 값을 리턴하는 함수를 만든다. 

def idx_of_outliers(x):
    q1, q3 = np.percentile(x, [25,75])
    IQR = q3 - q1
    lower_bound = q1 - [IQR * 1.5]
    upper_bound = q3 + [IQR * 1.5]
    return np.where((x > upper_bound) | (x < lower_bound))


In [4]:
idx_of_outliers(x.to_numpy())

(array([10], dtype=int64),)

백분위 수에서 5th ~ 95th 범위에서 벗어나는 경우를 outlier로 할 수도 있다.

In [5]:
print('5th percentile: ', x.quantile(q=0.05))
print('95th percentile: ', x.quantile(q=0.95))

5th percentile:  2.0
95th percentile:  61.5


In [6]:
x[(x < x.quantile(q=0.05)) | (x > x.quantile(q=0.95))]

1      1
10    78
dtype: int64

### outlier  처리하기
* outliers의 값을 제거
* 자연로그를 취해서 값을 감소시키는 방법 등으로 변환


In [7]:
houses = pd.DataFrame()
houses['Price'] = [534433, 392333, 293222, 4322032]
houses['Bedrooms'] = [2, 3.5, 2, 116]
houses['Square_Feets'] = [1500, 2500, 1500, 48000]
houses

,Price,Bedrooms,Square_Feets
0,534433,2.0,1500
1,392333,3.5,2500
2,293222,2.0,1500
3,4322032,116.0,48000


outlier를 처리하는 가장 간단한 방법은 outlier를 삭제하는 것이다.

In [8]:
houses.describe()

,Price,Bedrooms,Square_Feets
count,4.000000e+00,4.000000,4.000000
mean,1.385505e+06,30.875000,13375.000000
std,1.960186e+06,56.754405,23088.146309
min,2.932220e+05,2.000000,1500.000000
25%,3.675552e+05,2.000000,1500.000000
50%,4.633830e+05,2.750000,2000.000000
75%,1.481333e+06,31.625000,13875.000000
max,4.322032e+06,116.000000,48000.000000


In [9]:
q1 = houses['Bedrooms'].quantile(0.25)                 
q3 = houses['Bedrooms'].quantile(0.75)
iqr = q3 - q1
# Apply filter with respect to IQR
filter = (houses['Bedrooms'] >= q1 - 1.5*iqr) & (houses['Bedrooms'] <= q3 + 1.5*iqr)
houses.loc[filter] 

,Price,Bedrooms,Square_Feets
0,534433,2.0,1500
1,392333,3.5,2500
2,293222,2.0,1500


outlier의 영향이 줄어들도록 column을 변환한다.

In [10]:
# 로그변환
houses['Log_Square_Feets'] = [np.log(x) for x in houses['Square_Feets']]
houses

,Price,Bedrooms,Square_Feets,Log_Square_Feets
0,534433,2.0,1500,7.313220
1,392333,3.5,2500,7.824046
2,293222,2.0,1500,7.313220
3,4322032,116.0,48000,10.778956


### 결론
* outlier의 확인 및 처리 방법에는 정답이 없다. 
* 여러가지 방법을 고려하여 적절한 방법을 선택한다.
